# Agent Builder Framework Tutorial

Welcome to the comprehensive tutorial for the **Agent Builder Framework**! This notebook will guide you through:

- 🏗️ Setting up the framework
- 🤖 Creating and configuring agents
- 🔌 Working with plugins
- 📊 Monitoring and metrics
- 🛠️ Advanced usage patterns

## Prerequisites

Make sure you have:
1. Node.js 16+ installed
2. The agent-builder project built (`npm run build`)
3. Dependencies installed (`npm install`)

## 1. Framework Setup and Initialization

Let's start by importing the core components and setting up our agent engine.

In [ ]:
// First, let's import all the necessary components
const { AgentEngine } = require('./dist/core/engine');
const { ChatAgent, TaskAgent } = require('./dist/agents');
const { LoggingPlugin, CachePlugin, PluginLoader } = require('./dist/plugins');
const { ConfigLoader } = require('./dist/config');

// Initialize the core engine
const engine = new AgentEngine();
const configLoader = new ConfigLoader();
const pluginLoader = new PluginLoader();

console.log('✅ Agent Builder Framework initialized!');
console.log('Available agent types:', engine.getAvailableTypes());

## 2. Registering Agent Types

Before we can create agents, we need to register the available agent types with the engine.

In [ ]:
// Register our built-in agent types
engine.registerAgentType('chat', ChatAgent);
engine.registerAgentType('task', TaskAgent);

console.log('📝 Registered agent types:', engine.getAvailableTypes());

// Let's also set up some plugins
const loggingPlugin = new LoggingPlugin();
const cachePlugin = new CachePlugin(30000); // 30 second cache

pluginLoader.registerPlugin(loggingPlugin);
pluginLoader.registerPlugin(cachePlugin);

console.log('🔌 Loaded plugins:', pluginLoader.getPluginNames());

## 3. Creating Your First Chat Agent

Let's create a friendly chat agent that can respond to messages with different personalities.

In [ ]:
// Create a chat agent configuration
const chatConfig = {
  name: 'friendly-assistant',
  version: '1.0.0',
  description: 'A friendly conversational assistant',
  capabilities: ['chat', 'conversation', 'assistance'],
  settings: {
    personality: 'friendly',
    maxResponseLength: 150,
    language: 'en'
  },
  maxExecutionTime: 5000,
  retryPolicy: {
    maxRetries: 2,
    backoffMs: 1000
  }
};

// Create the chat agent
const chatAgent = await engine.createAgent('chat-1', 'chat', chatConfig);

console.log('🤖 Created chat agent:', chatAgent.getConfig().name);
console.log('Agent status:', chatAgent.getStatus());
console.log('Agent settings:', chatAgent.getSettings());

## 4. Executing the Chat Agent

Now let's interact with our chat agent by sending it some messages.

In [ ]:
// Execute the chat agent with different messages
const messages = [
  'Hello! How are you today?',
  'Can you help me understand AI agents?',
  'What makes you special?'
];

console.log('💬 Testing Chat Agent Responses:\n');

for (const message of messages) {
  console.log(`👤 User: ${message}`);
  
  const response = await engine.executeAgent('chat-1', {
    message: message
  }, {
    userId: 'tutorial-user',
    environment: 'notebook'
  });
  
  if (response.success) {
    console.log(`🤖 Agent: ${response.data.response}`);
    console.log(`⏱️  Execution time: ${response.executionTime}ms\n`);
  } else {
    console.log(`❌ Error: ${response.error}\n`);
  }
}

## 5. Testing Different Personalities

Let's see how the chat agent behaves with different personality settings.

In [ ]:
// Test different personalities
const personalities = ['professional', 'creative', 'friendly'];
const testMessage = 'Tell me about artificial intelligence.';

console.log('🎭 Testing Different Personalities:\n');

for (const personality of personalities) {
  // Get the chat agent instance and update its personality
  const agent = engine.getAgent('chat-1');
  agent.updateSettings({ personality });
  
  console.log(`🎯 Personality: ${personality.toUpperCase()}`);
  console.log(`👤 User: ${testMessage}`);
  
  const response = await engine.executeAgent('chat-1', {
    message: testMessage
  });
  
  if (response.success) {
    console.log(`🤖 Agent: ${response.data.response}\n`);
  }
}

## 6. Creating and Managing Task Agents

Now let's explore the TaskAgent, which can manage and execute tasks with priority scheduling.

In [ ]:
// Create a task agent configuration
const taskConfig = {
  name: 'task-manager',
  version: '1.0.0',
  description: 'Manages and executes automated tasks',
  capabilities: ['task-management', 'scheduling', 'automation'],
  settings: {
    maxConcurrentTasks: 3,
    defaultPriority: 'medium',
    autoSchedule: false // We'll manually execute tasks
  }
};

// Create the task agent
const taskAgent = await engine.createAgent('task-1', 'task', taskConfig);

console.log('📋 Created task agent:', taskAgent.getConfig().name);
console.log('Task agent settings:', taskAgent.getSettings());

## 7. Working with Task Management

Let's create, execute, and monitor tasks using our TaskAgent.

In [ ]:
// Create multiple tasks with different priorities
const tasksToCreate = [
  {
    action: 'create',
    name: 'Process user data',
    description: 'Clean and validate user input data',
    priority: 'high'
  },
  {
    action: 'create',
    name: 'Generate report',
    description: 'Create weekly analytics report',
    priority: 'medium'
  },
  {
    action: 'create',
    name: 'Backup database',
    description: 'Perform scheduled database backup',
    priority: 'low'
  }
];

console.log('📝 Creating Tasks:\n');
const createdTasks = [];

for (const taskData of tasksToCreate) {
  const response = await engine.executeAgent('task-1', taskData);
  
  if (response.success) {
    createdTasks.push(response.data.taskId);
    console.log(`✅ Created task: ${response.data.task.name} (${response.data.taskId})`);
    console.log(`   Priority: ${response.data.task.priority}`);
  }
}

console.log(`\n📊 Created ${createdTasks.length} tasks`);

In [ ]:
// List all tasks
const listResponse = await engine.executeAgent('task-1', {
  action: 'list'
});

if (listResponse.success) {
  console.log('📋 All Tasks:');
  listResponse.data.tasks.forEach((task, index) => {
    console.log(`${index + 1}. ${task.name}`);
    console.log(`   ID: ${task.id}`);
    console.log(`   Status: ${task.status}`);
    console.log(`   Priority: ${task.priority}`);
    console.log(`   Created: ${new Date(task.createdAt).toLocaleString()}\n`);
  });
}

## 8. Executing Tasks

Now let's execute some of the tasks we created and see the results.

In [ ]:
// Execute the first task (high priority)
console.log('🚀 Executing Tasks:\n');

for (const taskId of createdTasks.slice(0, 2)) { // Execute first 2 tasks
  console.log(`⚡ Executing task: ${taskId}`);
  
  const executeResponse = await engine.executeAgent('task-1', {
    action: 'execute',
    taskId: taskId
  });
  
  if (executeResponse.success) {
    console.log(`✅ Task completed successfully!`);
    console.log(`📊 Result:`, executeResponse.data.result);
    console.log(`⏱️  Execution time: ${executeResponse.executionTime}ms\n`);
  } else {
    console.log(`❌ Task failed: ${executeResponse.error}\n`);
  }
}

## 9. Agent Metrics and Monitoring

Let's examine the performance metrics that are automatically collected for our agents.

In [ ]:
// Get metrics for both agents
const chatAgentMetrics = engine.getAgent('chat-1').getMetrics();
const taskAgentMetrics = engine.getAgent('task-1').getMetrics();

console.log('📊 Agent Performance Metrics:\n');

console.log('🤖 Chat Agent Metrics:');
console.log(`   Executions: ${chatAgentMetrics.executionsCount}`);
console.log(`   Success Rate: ${chatAgentMetrics.successRate.toFixed(1)}%`);
console.log(`   Avg Execution Time: ${chatAgentMetrics.averageExecutionTime.toFixed(0)}ms`);
if (chatAgentMetrics.lastExecution) {
  console.log(`   Last Execution: ${chatAgentMetrics.lastExecution.toLocaleString()}`);
}

console.log('\n📋 Task Agent Metrics:');
console.log(`   Executions: ${taskAgentMetrics.executionsCount}`);
console.log(`   Success Rate: ${taskAgentMetrics.successRate.toFixed(1)}%`);
console.log(`   Avg Execution Time: ${taskAgentMetrics.averageExecutionTime.toFixed(0)}ms`);
if (taskAgentMetrics.errors.length > 0) {
  console.log(`   Recent Errors: ${taskAgentMetrics.errors.length}`);
}

// Get overall engine statistics
const engineStats = engine.getStats();
console.log('\n🏗️ Engine Statistics:');
console.log(`   Total Agents: ${engineStats.totalAgents}`);
console.log(`   Total Executions: ${engineStats.totalExecutions}`);
console.log(`   Agent Status Distribution:`, engineStats.agentsByStatus);

## 10. Working with Configuration Files

Let's demonstrate how to load agent configurations from external files.

In [ ]:
// Load configuration from the example files we created
try {
  const chatConfigFromFile = await configLoader.loadFromFile('./examples/chat-agent-config.json');
  console.log('📁 Loaded Chat Agent Config from JSON:');
  console.log(JSON.stringify(chatConfigFromFile, null, 2));
  
  const taskConfigFromFile = await configLoader.loadFromFile('./examples/task-agent-config.yaml');
  console.log('\n📁 Loaded Task Agent Config from YAML:');
  console.log(JSON.stringify(taskConfigFromFile, null, 2));
  
  // Create a new agent from the loaded configuration
  const fileBasedAgent = await engine.createAgent('chat-from-file', 'chat', chatConfigFromFile);
  console.log('\n✅ Created agent from file configuration!');
  
} catch (error) {
  console.error('❌ Error loading configuration files:', error.message);
  console.log('💡 Make sure to run this from the project root directory');
}

## 11. Plugin System Demonstration

Let's see how plugins enhance agent functionality by examining plugin statistics and behavior.

In [ ]:
// Check plugin metadata
const pluginMetadata = pluginLoader.getPluginMetadata();
console.log('🔌 Loaded Plugins:');
pluginMetadata.forEach(plugin => {
  console.log(`   ${plugin.name} v${plugin.version}`);
  if (plugin.description) {
    console.log(`   ${plugin.description}`);
  }
});

// Get cache plugin statistics
console.log('\n📈 Cache Plugin Statistics:');
const cacheStats = cachePlugin.getCacheStats();
console.log(`   Total Entries: ${cacheStats.totalEntries}`);
console.log(`   Valid Entries: ${cacheStats.validEntries}`);
console.log(`   Expired Entries: ${cacheStats.expiredEntries}`);

// Test cache functionality by executing the same request twice
console.log('\n⚡ Testing Cache Performance:');
const testMessage = 'What is machine learning?';

// First execution (should be cached)
const firstResponse = await engine.executeAgent('chat-1', { message: testMessage });
console.log(`First execution time: ${firstResponse.executionTime}ms`);

// Second execution (should be faster if cached)
const secondResponse = await engine.executeAgent('chat-1', { message: testMessage });
console.log(`Second execution time: ${secondResponse.executionTime}ms`);

if (secondResponse.executionTime < firstResponse.executionTime) {
  console.log('🚀 Cache improved performance!');
} else {
  console.log('📝 Cache may not have been used (new response generated)');
}

## 12. Error Handling and Retry Logic

Let's test the framework's error handling and retry capabilities.

In [ ]:
// Test error handling with invalid input
console.log('🧪 Testing Error Handling:\n');

// Test 1: Empty message to chat agent
console.log('Test 1: Empty message');
const emptyMessageResponse = await engine.executeAgent('chat-1', {
  message: ''
});

console.log(`Success: ${emptyMessageResponse.success}`);
if (!emptyMessageResponse.success) {
  console.log(`Error: ${emptyMessageResponse.error}`);
}

// Test 2: Invalid action for task agent
console.log('\nTest 2: Invalid task action');
const invalidActionResponse = await engine.executeAgent('task-1', {
  action: 'invalid-action'
});

console.log(`Success: ${invalidActionResponse.success}`);
if (!invalidActionResponse.success) {
  console.log(`Error: ${invalidActionResponse.error}`);
}

// Test 3: Non-existent agent
console.log('\nTest 3: Non-existent agent');
try {
  await engine.executeAgent('non-existent-agent', { test: true });
} catch (error) {
  console.log(`Caught error: ${error.message}`);
}

console.log('\n✅ Error handling tests completed!');

## 13. Advanced Features: Agent Events

Agents emit events during their lifecycle. Let's listen to these events for monitoring.

In [ ]:
// Set up event listeners for the chat agent
const chatAgentInstance = engine.getAgent('chat-1');

// Listen to execution events
chatAgentInstance.on('execution-start', (input) => {
  console.log('🚀 Execution started:', {
    type: input.type,
    sessionId: input.context.sessionId
  });
});

chatAgentInstance.on('execution-complete', (response) => {
  console.log('✅ Execution completed:', {
    success: response.success,
    executionTime: response.executionTime
  });
});

chatAgentInstance.on('execution-error', (response) => {
  console.log('❌ Execution error:', {
    error: response.error,
    executionTime: response.executionTime
  });
});

// Test the event system
console.log('🎯 Testing Agent Events:\n');
const eventTestResponse = await engine.executeAgent('chat-1', {
  message: 'Testing event system!'
});

console.log('\n📝 Event listeners are now active for future executions.');

## 14. Performance Comparison

Let's run a performance test to see how our agents perform under load.

In [ ]:
// Performance test: Execute multiple requests rapidly
console.log('⚡ Performance Test - Rapid Executions:\n');

const testMessages = [
  'What is AI?',
  'How do neural networks work?',
  'Explain machine learning',
  'What are the benefits of automation?',
  'How can AI help businesses?'
];

const startTime = Date.now();
const results = [];

// Execute all messages concurrently
const promises = testMessages.map(async (message, index) => {
  const response = await engine.executeAgent('chat-1', { message });
  results.push({
    index,
    message,
    success: response.success,
    executionTime: response.executionTime
  });
});

await Promise.all(promises);
const totalTime = Date.now() - startTime;

// Analyze results
const successfulExecutions = results.filter(r => r.success);
const averageExecutionTime = successfulExecutions.reduce((sum, r) => sum + r.executionTime, 0) / successfulExecutions.length;

console.log('📊 Performance Test Results:');
console.log(`   Total Requests: ${testMessages.length}`);
console.log(`   Successful: ${successfulExecutions.length}`);
console.log(`   Total Time: ${totalTime}ms`);
console.log(`   Average Execution Time: ${averageExecutionTime.toFixed(1)}ms`);
console.log(`   Requests per Second: ${((testMessages.length / totalTime) * 1000).toFixed(2)}`);

// Show individual results
console.log('\n📝 Individual Results:');
results.forEach(result => {
  console.log(`   ${result.index + 1}. ${result.success ? '✅' : '❌'} ${result.executionTime}ms - "${result.message}"`);
});

## 15. Cleanup and Final Statistics

Let's wrap up by showing final statistics and cleaning up our resources.

In [ ]:
// Final statistics summary
console.log('📈 Final Statistics Summary:\n');

const finalStats = engine.getStats();
console.log('🏗️ Engine Statistics:');
console.log(`   Total Agents Created: ${finalStats.totalAgents}`);
console.log(`   Total Executions: ${finalStats.totalExecutions}`);
console.log(`   Available Agent Types: ${finalStats.availableTypes.join(', ')}`);

// Individual agent final metrics
console.log('\n🤖 Final Agent Metrics:');
const agents = engine.getAllAgents();
agents.forEach((agent, agentId) => {
  const metrics = agent.getMetrics();
  const config = agent.getConfig();
  
  console.log(`\n   ${config.name} (${agentId}):`);
  console.log(`     Status: ${agent.getStatus()}`);
  console.log(`     Executions: ${metrics.executionsCount}`);
  console.log(`     Success Rate: ${metrics.successRate.toFixed(1)}%`);
  console.log(`     Avg Time: ${metrics.averageExecutionTime.toFixed(1)}ms`);
});

// Plugin statistics
console.log('\n🔌 Plugin Statistics:');
console.log(`   Loaded Plugins: ${pluginLoader.getAllPlugins().length}`);
const finalCacheStats = cachePlugin.getCacheStats();
console.log(`   Cache Entries: ${finalCacheStats.totalEntries} (${finalCacheStats.validEntries} valid)`);

console.log('\n🎉 Tutorial completed successfully!');
console.log('💡 You can now build your own agents using this framework!');

In [ ]:
// Cleanup: Shutdown the engine and clean up resources
console.log('🧹 Cleaning up resources...');

// Shutdown the engine (this will cleanup all agents)
await engine.shutdown();

// Unload all plugins
await pluginLoader.unloadAllPlugins();

console.log('✅ Cleanup completed!');
console.log('\n🎓 Thank you for completing the Agent Builder Framework tutorial!');
console.log('📚 Next steps:');
console.log('   - Create custom agent types by extending the Agent base class');
console.log('   - Build custom plugins using the BasePlugin interface');
console.log('   - Explore the CLI tools with `npx agent-builder --help`');
console.log('   - Check out the examples/ directory for more use cases');

## 🎯 Summary

Congratulations! You've completed the comprehensive Agent Builder Framework tutorial. Here's what we covered:

### ✅ Key Concepts Learned:
1. **Framework Setup** - Initializing the AgentEngine and registering agent types
2. **Agent Creation** - Creating ChatAgent and TaskAgent with custom configurations
3. **Agent Execution** - Running agents with different inputs and contexts
4. **Configuration Management** - Loading configs from JSON/YAML files
5. **Plugin System** - Using LoggingPlugin and CachePlugin for enhanced functionality
6. **Monitoring & Metrics** - Tracking performance, success rates, and execution times
7. **Error Handling** - Understanding retry policies and error recovery
8. **Event System** - Listening to agent lifecycle events
9. **Performance Testing** - Measuring throughput and concurrent execution

### 🚀 Next Steps:
- **Custom Agents**: Extend the `Agent` base class to create specialized agents
- **Custom Plugins**: Build plugins using the `BasePlugin` interface
- **Production Deployment**: Use the CLI tools for production management
- **Integration**: Integrate agents into larger applications

### 📖 Resources:
- [Project README](./README.md) - Comprehensive documentation
- [API Reference](./.github/copilot-instructions.md) - Detailed API guide
- [Examples Directory](./examples/) - More usage examples
- [CLI Documentation](./src/cli.ts) - Command-line interface

Happy building! 🎉